In [1]:
import sys, os
sys.path.append(os.getcwd() + '\..')
import pandas as pd
import numpy as np
from my_api import *
import matplotlib.pyplot as plt

0.9
12.685905874921438 8.685905874921438
0.5
15.15749916396364 11.15749916396364


d:\BaiduSyncdisk\Project\Co_simu_3\code_Cai2011\..\gen_S.py:141: ComplexWarning: Casting complex values to real discards the imaginary part
  S = S.astype(np.float64)


In [2]:
id = pd.MultiIndex.from_product([['fro', 2], [100, 300, 500]])
df = pd.DataFrame(index = id)

In [3]:
T = 300
rela = False
cov_str = 'Cai2011Adaptive_Model2_my'

In [5]:
simu_str = 'lx_band'
data_path = '../data_Cai2_Bernoulli/'
for N in [100, 300, 500]:
    if cov_str == 'Cai2011Adaptive_Model1':
        S = gen_S_Cai2011Adaptive_Model1(N)
    elif cov_str == 'Cai2011Adaptive_Model2_my':
        S = gen_S_Cai2011Adaptive_Model2_my(N = N, seed = 0, probB = 10 / (N // 2))
    else:
        raise Exception
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)
        
        for eta in [0.5, 0.8, 1]:
            cov_params = [cov_str]
            simu_params = [simu_str, eta]
            file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
            file_name = data_path + file_name
            data = np.loadtxt(file_name)

            col = str(simu_params)
            if col not in df.columns:
                df[col] = None
            if not rela:
                data *= normS # absolute error
            df[col][ord, N] = f"{data.mean() :.2f}({data.std() :.2f})"
df

['lx_band', 0.5] ['lx_band', 0.8] ['lx_band', 1]
fro 100      14.69(0.22)      10.88(0.27)     9.95(0.26)
    300      24.30(0.28)      18.52(0.35)    16.91(0.31)
    500      30.03(0.19)      22.10(0.29)    20.39(0.30)
2   100       5.13(0.35)       3.61(0.35)     3.52(0.28)
    300       5.09(0.22)       3.66(0.17)     3.66(0.15)
    500       4.58(0.16)       4.04(0.14)     3.97(0.17)

In [6]:
simu_str = 'LSRthreshold'
data_path = '../data_Cai2_Bernoulli/'
num_cv = 50

for N in [100, 300, 500]:
    if cov_str == 'Cai2011Adaptive_Model1':
        S = gen_S_Cai2011Adaptive_Model1(N)
    elif cov_str == 'Cai2011Adaptive_Model2_my':
        S = gen_S_Cai2011Adaptive_Model2_my(N = N, seed = 0, probB = 10 / (N // 2))
    else:
        raise Exception
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)
        
        for tau in [0.2]:
            # for prob in [0.99, 0.9, 1]:
            #     for qrob in [0, 0.01, 0.1]:
            for prob in [1, 0.99]:
                for qrob in [0, 0.01]:
                    cov_params = [cov_str]
                    simu_params = [simu_str, tau, prob, qrob, 'brute', num_cv]
                    file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
                    file_name = data_path + file_name
                    
                    try:
                        data = np.loadtxt(file_name)
                    except FileNotFoundError:
                        continue

                    col = str(simu_params)
                    if col not in df.columns:
                        df[col] = None
                    if not rela:
                        data *= normS # absolute error
                        df[col][ord, N] = f"{data.mean() :.2f}({data.std() :.2f})"
df

['lx_band', 0.5] ['lx_band', 0.8] ['lx_band', 1]  \
fro 100      14.69(0.22)      10.88(0.27)     9.95(0.26)   
    300      24.30(0.28)      18.52(0.35)    16.91(0.31)   
    500      30.03(0.19)      22.10(0.29)    20.39(0.30)   
2   100       5.13(0.35)       3.61(0.35)     3.52(0.28)   
    300       5.09(0.22)       3.66(0.17)     3.66(0.15)   
    500       4.58(0.16)       4.04(0.14)     3.97(0.17)   

        ['LSRthreshold', 0.2, 1, 0, 'brute', 50]  
fro 100                              10.35(1.65)  
    300                              26.82(4.30)  
    500                              36.56(6.83)  
2   100                               3.01(0.61)  
    300                               5.00(0.80)  
    500                               5.26(1.04)

In [9]:
simu_str = '' # to be determined
data_path = '../data_Cai2_Bernoulli/'

for N in [100, 300, 500]:
    if cov_str == 'Cai2011Adaptive_Model1':
        S = gen_S_Cai2011Adaptive_Model1(N)
    elif cov_str == 'Cai2011Adaptive_Model2_my':
        S = gen_S_Cai2011Adaptive_Model2_my(N = N, seed = 0, probB = 10 / (N // 2))
    else:
        raise Exception
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)

        # from Cai2011_other import Other_Methods
        for simu_str in ['Sample', 'Soft Threshold', 'Hard Threshold', 'Linear Shrink', 'Nonlinear Shrink']: 
            cov_params = [cov_str]
            simu_params = [simu_str]
            file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
            file_name = data_path + file_name
            try:
                data = np.loadtxt(file_name)
            except FileNotFoundError:
                continue

            col = str(*simu_params)
            if col not in df.columns:
                df[col] = None
            
            if not rela:
                data *= normS # absolute error
            df[col][ord, N] = f"{data.mean() :.2f}({data.std() :.2f})"
df

['lx_band', 0.5] ['lx_band', 0.8] ['lx_band', 1]  \
fro 100      14.69(0.22)      10.88(0.27)     9.95(0.26)   
    300      24.30(0.28)      18.52(0.35)    16.91(0.31)   
    500      30.03(0.19)      22.10(0.29)    20.39(0.30)   
2   100       5.13(0.35)       3.61(0.35)     3.52(0.28)   
    300       5.09(0.22)       3.66(0.17)     3.66(0.15)   
    500       4.58(0.16)       4.04(0.14)     3.97(0.17)   

        ['LSRthreshold', 0.2, 1, 0, 'brute', 50] Soft Threshold  \
fro 100                              10.35(1.65)    19.03(0.52)   
    300                              26.82(4.30)    53.95(0.70)   
    500                              36.56(6.83)    82.93(0.71)   
2   100                               3.01(0.61)     5.00(0.41)   
    300                               5.00(0.80)     9.58(0.51)   
    500                               5.26(1.04)    12.25(0.25)   

        Hard Threshold        Sample Linear Shrink Nonlinear Shrink  
fro 100    25.71(0.51)   25.94(0.49)   16.26(0.27)      15.20(0.32)  
    300    81.89(0.68)   83.47(0.63)   33.76(0.10)             None  
    500   133.60(0.66)  137.19(0.60)   44.59(0.07)             None  
2   100     6.98(0.59)    7.05(0.59)    6.82(0.66)       5.67(0.89)  
    300    15.06(0.63)   15.48(0.66)    9.08(0.23)             None  
    500    20.58(0.36)   21.43(0.38)    9.41(0.13)             None

In [10]:
df

['lx_band', 0.5] ['lx_band', 0.8] ['lx_band', 1]  \
fro 100      14.69(0.22)      10.88(0.27)     9.95(0.26)   
    300      24.30(0.28)      18.52(0.35)    16.91(0.31)   
    500      30.03(0.19)      22.10(0.29)    20.39(0.30)   
2   100       5.13(0.35)       3.61(0.35)     3.52(0.28)   
    300       5.09(0.22)       3.66(0.17)     3.66(0.15)   
    500       4.58(0.16)       4.04(0.14)     3.97(0.17)   

        ['LSRthreshold', 0.2, 1, 0, 'brute', 50] Soft Threshold  \
fro 100                              10.35(1.65)    19.03(0.52)   
    300                              26.82(4.30)    53.95(0.70)   
    500                              36.56(6.83)    82.93(0.71)   
2   100                               3.01(0.61)     5.00(0.41)   
    300                               5.00(0.80)     9.58(0.51)   
    500                               5.26(1.04)    12.25(0.25)   

        Hard Threshold        Sample Linear Shrink Nonlinear Shrink  
fro 100    25.71(0.51)   25.94(0.49)   16.26(0.27)      15.20(0.32)  
    300    81.89(0.68)   83.47(0.63)   33.76(0.10)             None  
    500   133.60(0.66)  137.19(0.60)   44.59(0.07)             None  
2   100     6.98(0.59)    7.05(0.59)    6.82(0.66)       5.67(0.89)  
    300    15.06(0.63)   15.48(0.66)    9.08(0.23)             None  
    500    20.58(0.36)   21.43(0.38)    9.41(0.13)             None

In [11]:
new_cols = []
for col in df.columns:
    try:
        col = eval(col)
    except Exception:
        pass

    if type(col) == str:
        new_col = col
    elif 'lx_band' in col:
        eta = col[1]
        new_col = fr'Augmented Band ($\eta={eta}$)'
    elif 'LSRthreshold' in col or 'thG' in col:
        formal_name = {'LSRthreshold': 'Augmented Threshold', 'thG': 'Threshold then Augment'}[col[0]]
        t, p, q = col[1:-2]
        new_col = fr'{formal_name} ($\tau={t}, p={p}, q={q}$)' # The r in fr, means raw string. Otherwise I should write \\tau instead of \tau.
    new_cols.append(new_col)
df.columns = new_cols
df.head(1)

,,Augmented Band ($\eta=0.5$),Augmented Band ($\eta=0.8$),Augmented Band ($\eta=1$),"Augmented Threshold ($\tau=0.2, p=1, q=0$)",Soft Threshold,Hard Threshold,Sample,Linear Shrink,Nonlinear Shrink
fro,100,14.69(0.22),10.88(0.27),9.95(0.26),10.35(1.65),19.03(0.52),25.71(0.51),25.94(0.49),16.26(0.27),15.20(0.32)


In [12]:
buf = '../latex/' + cov_str + '.tex'
caption = cov_str

with pd.option_context("max_colwidth", 1000, 'display.width', 50):
    df.to_latex(buf = buf, 
                index = True, na_rep = ' ', sparsify = True, 
             column_format = 'll|' + 'p{2cm}' * df.shape[1], 
             longtable = False, 
             escape = False, encoding = 'utf-8', 
             caption = caption, label = "my label", 
             position = 'htbp')
# escape参数，防止字符被转换使得我设的数学模式被破坏

C:\Users\lx\AppData\Local\Temp\ipykernel_6364\2522360471.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex(buf = buf,


In [13]:
pd.get_option('display.width')

80

In [14]:
lines = []

with open(buf, 'r') as fp:
    lines = fp.readlines()

s1 = r'\resizebox{\linewidth}{!}{' + '\n'
s2 = r'}' + '\n'
lines.insert(4, s1)
lines.insert(-1, s2)

with open(buf[:-4] + '_processed' + '.tex', 'w') as fp:
    fp.writelines(lines)
        # if not write_flag:
        #     if r'begin{tabular}' in line:
